In [49]:
from bs4 import BeautifulSoup
import grequests
import requests
from multiprocessing import Pool
import numpy as np

In [50]:
def strip_white_chars(text):
    return text.strip("\n").strip(" ")

In [66]:
def get_product_data(product_request):
    """Function is responsible for scraping product data from 4U.am product content
    """
    try:
        soup = BeautifulSoup(product_request.content, 'html.parser')
        sp = soup.find_all("div", {"class": "info_container"})[0]

        product_data = {}
        product_data["product"] = sp.find('p', attrs={'class' : 'name_info'}).text

        brand_partner = sp.find_all('div', attrs={'class' : 'brand_partner'})
        product_data["item_code"] = brand_partner[0].span.strong.text
        product_data["store"] = strip_white_chars(brand_partner[1].find_all("span")[0].a.text)
        try:
            product_data["brand"] = strip_white_chars(brand_partner[1].find_all("span")[1].a.text)
        except:
            pass
        product_data["url"] = product_request.url
        product_data["price"] = strip_white_chars(sp.find("p", attrs={'class' : 'item_price'}).span.span.text)
        product_data["item description"] = strip_white_chars(sp.find("div", attrs={'id' : 'description'}).div.text)
        product_data["image url"] = soup.find("div", attrs={'class' : 'show_tumb'}).img["src"]

        return product_data
    except:
        return dict()

In [67]:
def get_category_names():
    r = requests.get("https://4u.am/hy/")
    soup = BeautifulSoup(r.content, 'html.parser')
    text = str(soup)
    return list(set(re.findall(r"\/hy/category/([\w-]+)[\?|\"]\b", text)))

In [68]:
def extract_category_contents(category_names):
    rs = (grequests.get("https://4u.am/hy/category/"+cat) for cat in category_names)
    return grequests.map(rs)

def extract_item_contents(item_urls):
    rs = (grequests.get("https://4u.am"+item) for item in item_urls)
    return grequests.map(rs)

In [69]:
def extract_item_urls(category_request):
    items = []
    soup = BeautifulSoup(category_request.content, 'html.parser')
    sp= soup.find_all('div', attrs={'class' : 'category_item'})
    for i in sp: items.append(i.find_all("a")[1]["href"])
    return items

In [70]:
def extract_item_info(category_names):
    
    with Pool(5) as p:
        item_urls = p.map(extract_item_urls, extract_category_contents(category_names), chunksize=1)
    urls = [j for i in item_urls for j in i]

    with Pool(5) as p:
        result = p.map(get_product_data,extract_item_contents(urls))
    
    return result

In [71]:
category_names =  get_category_names()

In [73]:
test = extract_item_info(category_names)

https://4u.am/hy/product/vardajur-nairian-micelayin
https://4u.am/hy/product/pucikner-luchiya-no11
https://4u.am/hy/product/sampun-inferno-tepi-dem
https://4u.am/hy/product/multheros-star-wars-mandalorian-the-child-pram
https://4u.am/hy/product/danak-kizlyar-pogpate-no5
https://4u.am/hy/product/tegapoxman-vandak-beeztees-ktore
https://4u.am/hy/product/girk-bar-ow-bani-stemaran
https://4u.am/hy/product/donat-kompozicia-siena
https://4u.am/hy/product/nver-kart-quest-project-isolation-la-casa-de-papel
https://4u.am/hy/product/banan-1-kg-5
https://4u.am/hy/product/ocanelik-erato-femme-passionnee
https://4u.am/hy/product/zartucic-creative-gifts-atrcanak
https://4u.am/hy/product/krem-skrab-nuard-dastayin-cagikner-2
https://4u.am/hy/product/sarf-moreni-ayb-erknaguyn
https://4u.am/hy/product/kaxazard-u-sgta-oskya-14
https://4u.am/hy/product/ekler-sweet-m
https://4u.am/hy/product/nver-tup-the-box-no224
https://4u.am/hy/product/kotletner-bellisimo-kievyan-1-kg
https://4u.am/hy/product/tort-muran

AttributeError: 'NoneType' object has no attribute 'url'

In [ ]:
import pickle
with open("data.pkl", "wb") as f:
    pickle.dump(test,f)

In [111]:
with open("data.pkl", "rb") as f:
    qq = pickle.load(f)